# Utils

> TODO fill in description

In [1]:
#| default_exp utils

In [2]:
#| hide
from nbdev.showdoc import *; 

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
from __future__ import annotations
import asyncio
from typing import Optional, Type, Union, Coroutine, List, Callable, Any, Tuple
from types import MappingProxyType
import copy
import traceback
import inspect
import re
import ast
from pathlib import Path
import subprocess
import os, sys
import importlib.util
import keyword
from abc import ABC, abstractmethod

import fbdev

In [5]:
#|hide
show_doc(fbdev.utils.is_mutually_exclusive)

---

### is_mutually_exclusive

>      is_mutually_exclusive (*args, at_least_one:bool=False)

In [6]:
#|export
def is_mutually_exclusive(*args, at_least_one:bool=False):
    num_not_None = sum(map(lambda a: not a is None, args))
    
    if at_least_one: return num_not_None == 1
    else: return num_not_None <= 1

In [7]:
assert is_mutually_exclusive(1)
assert is_mutually_exclusive(1, None)
assert not is_mutually_exclusive(1, None, 2)
assert not is_mutually_exclusive(None, None, None, at_least_one=True)
assert is_mutually_exclusive(None, 1, None, at_least_one=True)

In [8]:
#|hide
show_doc(fbdev.utils.is_in_event_loop)

---

### is_in_event_loop

>      is_in_event_loop ()

In [9]:
#|export
def is_in_event_loop():
    try:
        asyncio.get_running_loop()
        return True
    except RuntimeError:
        return False

In [10]:
#|hide
show_doc(fbdev.utils.await_multiple_events)

---

### await_multiple_events

>      await_multiple_events (*events)

In [11]:
#|export
async def await_multiple_events(*events):
    try:
        event_await_tasks = []
        while not all([event.is_set() for event in events]): # In the off-chance that as asyncio.wait finishes, one of the events is cleared
            event_await_tasks.clear()
            for event in events:
                event_await_tasks.append(asyncio.create_task(event.wait()))
            await asyncio.wait(event_await_tasks)
    except asyncio.CancelledError:
        for task in event_await_tasks:
            task.cancel()
            try: await task
            except asyncio.CancelledError: pass
        raise

In [12]:
#|hide
show_doc(fbdev.utils.await_any_event)

---

### await_any_event

>      await_any_event (*events)

In [13]:
#|export
async def await_any_event(*events):
    try:
        event_await_tasks = [asyncio.create_task(event.wait()) for event in events]
        await asyncio.wait(event_await_tasks, return_when=asyncio.FIRST_COMPLETED)
    except asyncio.CancelledError:
        for task in event_await_tasks:
            task.cancel()
            try: await task
            except asyncio.CancelledError: pass
        raise

In [14]:
ev1 = asyncio.Event()
async def foo1():
    await asyncio.sleep(0.5)
    ev1.set()
    
ev2 = asyncio.Event()
async def foo2():
    await asyncio.sleep(1)
    ev2.set()

asyncio.create_task(foo1())
asyncio.create_task(foo2())
await await_any_event(ev1, ev2)
print("Done waiting")
print("ev1 is set:", ev1.is_set())
print("ev2 is set:", ev2.is_set())

Done waiting
ev1 is set: True
ev2 is set: False


In [15]:
#|hide
show_doc(fbdev.utils.AttrContainer)

---

### AttrContainer

>      AttrContainer (_attrs=None, obj_name='AttrContainer',
>                     dtype:Optional[Type]=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [16]:
#|export
class AttrContainer:
    def __init__(self, _attrs=None, obj_name="AttrContainer", dtype:Optional[Type]=None):
        self.idx = ()
        self._attrs = dict(_attrs) if _attrs is not None else {}
        self._obj_name = obj_name
        self._dtype = dtype
        
    def __getattr__(self, key):
        if key.startswith("__") and key.endswith("__"):
            raise AttributeError(f"'{type(self).__name__}' object has no attribute '{key}' (in {self._obj_name})")
        return self[key]
        
    def __getitem__(self, key):
        if key in self._attrs:
            return self._attrs[key]
        elif key.startswith("__") and key.endswith("__"):
            raise KeyError(f"'{type(self).__name__}' object has no key '{key}' (in {self._obj_name})")
        else:
            raise Exception(f"'{key}' does not exist (in {self._obj_name})")
        
    def _set(self, key, value):
        if self._dtype is not None and type(value) != self._dtype:
            raise TypeError(f"Value {value} is not of type {self._dtype} (in {self._obj_name}).")
        self._attrs[key] = value
        
    def _remove(self, key):
        del self._attrs[key]
        
    def keys(self):
        return self._attrs.keys()
    
    def values(self):
        return self._attrs.values()
    
    def items(self):
        return self._attrs.items()
    
    def as_readonly_dict(self):
        return MappingProxyType(self._attrs)
    
    def as_list(self):
        return list(self.values())
        
    def __iter__(self):
        return self._attrs.__iter__()
    
    def __contains__(self, key):
        return key in self._attrs

    def __len__(self):
        return self._attrs.__len__()
    
    def __str__(self):
        return f'{self._obj_name}: {", ".join([f"{k}: {v}" for k,v in self._attrs.items()])}'
    
    def __repr__(self):
        return self.__str__()
    
    def copy(self):
        copy =  copy.copy(self)
        for key, value in self.items():
            if type(value) == AttrContainer:
                copy._set(key, value.copy())
        return copy

In [17]:
#|hide
show_doc(fbdev.utils.ReadonlyEvent)

---

### ReadonlyEvent

>      ReadonlyEvent (event:asyncio.Event)

*Initialize self.  See help(type(self)) for accurate signature.*

In [18]:
#|export
class ReadonlyEvent:
    def __init__(self, event: asyncio.Event):
        self._event = event

    def is_set(self):
        return self._event.is_set()

    async def wait(self):
        await self._event.wait()

In [19]:
#|hide
show_doc(fbdev.utils.EventHandler)

---

### EventHandler

>      EventHandler (name)

*Subscribable events*

In [20]:
#|export
class EventHandler:
    """Subscribable events"""
    def __init__(self, name):
        self._events = []
        self.name = name
    
    def subscribe(self):
        event = asyncio.Event()
        self._events.append(event)
        return event

    def _trigger(self):
        for event in self._events:
            event.set()
        self._events.clear()
        
    def __str__(self):
        return f"EventHandler(name='{self.name}')"
    
    def __repr__(self):
        return str(self)

In [21]:
#|hide
show_doc(fbdev.utils.EventCollection)

---

### EventCollection

>      EventCollection ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [22]:
#|export
class EventCollection(AttrContainer):
    def __init__(self) -> None:
        super().__init__({}, obj_name="EventCollection")
    
    def _add_event(self, event_handler: EventHandler):
        self._set(event_handler.name, event_handler)

In [23]:
#|hide
show_doc(fbdev.utils.StateHandler)

---

### StateHandler

>      StateHandler (name, current_state, state_vals=[True, False])

*Initialize self.  See help(type(self)) for accurate signature.*

In [24]:
#|export
class StateHandler:
    def __init__(self, name, current_state, state_vals=[True, False]):
        self.name = name
        state_vals = list(state_vals) # Can be enums
        self._state_vals = state_vals
        if len(state_vals) != len(set(state_vals)): raise ValueError("`state_vals` must have all unique elements.")
        if current_state not in state_vals: raise ValueError("`current_state` must be in `state_vals`.")
        self.__state_is_on = {state : asyncio.Event() for state in state_vals}
        self.__state_is_on[current_state].set()
        self.__state_is_off = {state : asyncio.Event() for state in state_vals}
        self._current_state = current_state
        for state in self.__state_is_off:
            if state != current_state: self.__state_is_off[state].set()
        
    def check(self, state):
        return self.__state_is_on[state].is_set()
    
    def get(self):
        return self._current_state
    
    def set(self, state):
        if state not in self._state_vals: raise ValueError(f"Invalid state: {state}. Possible states: {', '.join(self._state_vals)}")
        self._current_state = state
        for _state in self.__state_is_on:
            if _state == state:
                self.__state_is_on[_state].set()
                self.__state_is_off[_state].clear()
            else:
                self.__state_is_on[_state].clear()
                self.__state_is_off[_state].set()
            
    def wait(self, state, target_value=True):
        if target_value: return self.__state_is_on[state].wait()
        else: return self.__state_is_off[state].wait()
        
    async def __event_func(self, state, event):
        await state.wait()
        event.set()
      
    def get_state_event(self, state, target_value=True):
        if target_value: return ReadonlyEvent(self.__state_is_on[state])
        else: return ReadonlyEvent(self.__state_is_off[state])
        
    def get_state_toggle_event(self, state, target_value=True):
        event = asyncio.Event()
        if target_value: asyncio.create_task(self.__event_func(self.__state_is_on[state], event))
        else: asyncio.create_task(self.__event_func(self.__state_is_off[state], event))
        return event
    
    def __str__(self):
        return f"State {self.name}: {self._current_state}"
    
    def __repr__(self):
        return self.__str__()

In [25]:
#|hide
show_doc(fbdev.utils.StateView)

---

### StateView

>      StateView (state_handler)

*Initialize self.  See help(type(self)) for accurate signature.*

In [26]:
#|export
class StateView:
    def __init__(self, state_handler):
        self._state_handler: StateHandler = state_handler
        
    def check(self, state):
        return self._state_handler.check(state)
    
    def get(self):
        return self._state_handler._current_state
            
    def wait(self, state, state_value=True):
        return self._state_handler.wait(state, state_value)
      
    def get_state_event(self, state, state_value=True):
        return self._state_handler.get_state_event(state, state_value)
        
    def get_state_toggle_event(self, state, state_value=True):
        return self._state_handler.get_state_toggle_event(state, state_value)
    
    def __str__(self):
        return str(self._state_handler)
    
    def __repr__(self):
        return self.__str__()

In [27]:
#|hide
show_doc(fbdev.utils.StateCollection)

---

### StateCollection

>      StateCollection ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [28]:
#|export
class StateCollection(AttrContainer):
    def __init__(self) -> None:
        super().__init__({}, obj_name="StateCollection")
    
    def _add_state(self, state_handler: StateHandler, readonly=True):
        self._set(f"_{state_handler.name}", state_handler)
        if readonly:
            self._set(f"{state_handler.name}", StateView(state_handler))
        else:
            self._set(state_handler.name, state_handler)

In [29]:
#|hide
show_doc(fbdev.utils.TaskManager)

---

### TaskManager

>      TaskManager (host)

*Initialize self.  See help(type(self)) for accurate signature.*

In [30]:
#|export
class TaskManager:
    def __init__(self, host):
        self._host = host
        self._tasks = []
        self._cancelled_tasks = []
        self._monitoring_task = asyncio.create_task(self._monitor_tasks())
        self._callbacks: List[Callable[[asyncio.Task, Exception], None]] = []
        self._registered_exceptions = []
        self._exceptions_non_empty_condition = asyncio.Condition()
        self._tasks_non_empty_condition = asyncio.Condition()  # New condition for task management
        
    def create_task(self, coroutine:Coroutine) -> asyncio.Task:
        task = asyncio.create_task(coroutine)
        self._tasks.append(task)
        async def _notify():
            async with self._tasks_non_empty_condition:
                self._tasks_non_empty_condition.notify_all()
        asyncio.create_task(_notify())
        return task
        
    async def _monitor_tasks(self):
        try:
            while True:
                await asyncio.sleep(0)
                async with self._tasks_non_empty_condition:
                    await self._tasks_non_empty_condition.wait_for(lambda: len(self._tasks) > 0) 
                done, pending = await asyncio.wait(self._tasks, return_when=asyncio.FIRST_COMPLETED)
                for task in done:
                    try:
                        exception = task.exception()
                    except asyncio.CancelledError as e:
                        exception = e
                    if exception is not None:
                        async with self._exceptions_non_empty_condition:
                            self.submit_exception(task, exception, ())
                    self._tasks.remove(task)
        except asyncio.CancelledError: # This registers the task cancel exception as handled.
            pass
        except Exception as e:
            self.submit_exception(asyncio.current_task(), e, ())
                
    async def wait_for_exceptions(self):
        try:
            async with self._exceptions_non_empty_condition:
                await self._exceptions_non_empty_condition.wait_for(lambda: len(self._registered_exceptions) > 0)
        except asyncio.CancelledError: pass
                
    def exception_occured(self): return len(self._registered_exceptions) > 0
    
    def get_exceptions(self):
        return list(self._registered_exceptions)
                
    def cancel(self, task, msg=None):
        if task not in self._tasks:
            raise ValueError(f"Task {task} is not in the task manager.")
        if task in self._cancelled_tasks:
            raise ValueError(f"Task {task} is already cancelled.")
        task.cancel(msg)
        current_frame = inspect.currentframe()
        caller_name = current_frame.f_back.f_code.co_name #TODO potentially use for logging
        self._cancelled_tasks.append(task)
        
    async def cancel_wait(self, task, msg=None):
        self.cancel(task, msg)
        try:
            await asyncio.wait_for(task, timeout=None) 
        except asyncio.CancelledError:
            pass  # Task was cancelled successfully
        
    async def stop(self):
        for task in self._tasks:
            await self.cancel_wait(task)
        
    def is_cancelled(self, task:asyncio.Task):
        return task in self._cancelled_tasks
                
    def subscribe(self, callback: Callable[[asyncio.Task, Exception, Tuple[Any, ...]], None]):
        self._callbacks.append(callback)
        
    def submit_exception(self, task:asyncio.Task, exception:Exception, source_trace:Tuple[Any, ...]):
        self._registered_exceptions.append((task, exception, source_trace + (self._host,)))
        for callback in self._callbacks:
            callback(task, exception, source_trace + (self._host,))
        async def _notify():
            async with self._exceptions_non_empty_condition:
                self._exceptions_non_empty_condition.notify_all()
        asyncio.create_task(_notify())
            
    async def destroy(self):
        for task in self._tasks:
            await self.cancel_wait(task)
        self._monitoring_task.cancel()
        try: await self._monitoring_task
        except asyncio.CancelledError: pass
            
    def get_task_coro_qualnames(self):
        qualnames = [task.get_coro().__qualname__ for task in self._tasks]
        qualname_counts = {name : qualnames.count(name) for name in set(qualnames)}
        return qualname_counts

In [31]:
#|hide
show_doc(fbdev.utils.AddressableMixin)

---

### AddressableMixin

>      AddressableMixin ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [32]:
#|export
class AddressableMixin(ABC):
    __address_separators = {}
    
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        AddressableMixin.__address_separators[cls._address_separator] = cls.__name__
    
    @property
    @abstractmethod
    def id(self): raise NotImplementedError("Must be implemented by subclass.")
    
    @property
    def parent(self): raise NotImplementedError("Must be implemented by subclass.")
    
    @property
    def _children(self) -> List[AddressableMixin]: raise NotImplementedError("Must be implemented by subclass.")
    
    @property
    @abstractmethod
    def rich_id(self): raise NotImplementedError("Must be implemented by subclass.")
       
    @property
    def rich_address(self):
        return f"{self.parent.rich_address if self.parent is not None else ''}{self._address_separator}{self.rich_id}"
        
    @property
    def address(self):
        return f"{self.parent.address if self.parent is not None else ''}{self._address_separator}{self.id}"
    
    @classmethod
    def _validate_id(cls, id:str):
        if type(id) != str:
            raise TypeError(f"Id must be a string, not {type(id)}.")
        if any(c in id for c in cls.__address_separators.keys()):
            raise ValueError(f"Id cannot contain any of {set(cls.__address_separators.keys())}.")
        
    @classmethod
    def iter_address(cls, address:str):
        seps = list(cls.__address_separators.keys())
        _address = address
        while True:
            if _address[0] not in seps: raise ValueError(f"Poorly formatted address: '{address}'.")
            elem_type = cls.__address_separators[_address[0]]
            r_index = min([_address[1:].index(sep) for sep in seps if sep in _address[1:]] + [len(_address)])
            yield elem_type, _address[1:][:r_index]
            _address = _address[1:][r_index:]
            if not _address: break
            

In [33]:
#|hide
show_doc(fbdev.utils.get_git_root_directory)

---

### get_git_root_directory

>      get_git_root_directory ()

In [34]:
#|export
def get_git_root_directory():
    try:
        # Run 'git rev-parse --show-toplevel' command
        git_root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel'], stderr=subprocess.STDOUT).decode('utf-8').strip()
        return git_root
    except subprocess.CalledProcessError:
        # If the command fails, it means we're not in a git repository
        return None

In [35]:
get_git_root_directory()

'/Users/lukastk/proj_dev/fbdev'

In [36]:
cwd = os.getcwd()
os.chdir(os.path.expanduser("~"))
print(get_git_root_directory() is None)
os.chdir(cwd)

True


In [37]:
#|hide
show_doc(fbdev.utils.root_dir)

---

### root_dir

>      root_dir (module)

In [38]:
#|export
def root_dir(module):
    return Path(module.__file__).parent

In [39]:
root_dir(fbdev)

Path('/Users/lukastk/proj_dev/fbdev/fbdev')

In [40]:
#|hide
show_doc(fbdev.utils.extract_top_level_docstring)

---

### extract_top_level_docstring

>      extract_top_level_docstring (file_path)

In [41]:
#|export
def extract_top_level_docstring(file_path):
    with open(file_path, "r") as file:
        file_content = file.read()
    
    # Parse the file content using AST
    parsed_ast = ast.parse(file_content)
    
    # Retrieve the docstring of the module
    docstring = ast.get_docstring(parsed_ast)
    
    return docstring

In [42]:
get_git_root_directory()

'/Users/lukastk/proj_dev/fbdev'

In [43]:
# Example usage
file_path = Path(get_git_root_directory()) / "fbdev" / "dev.py"
docstring = extract_top_level_docstring(file_path)
if docstring:
    print("Top-level docstring:", docstring)
else:
    print("No top-level docstring found.")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lukastk/proj_dev/fbdev/fbdev/dev.py'

In [44]:
#|hide
show_doc(fbdev.utils.find_module_root)

---

### find_module_root

>      find_module_root (path)

In [45]:
#|exporti
def find_module_root(path):
    path = Path(path)
    path = path if path.is_dir() else path.parent
    is_module = '__init__.py' in [p.parts[-1] for p in path.glob('*')]
    if not is_module: return None
    else:
        parent_module = find_module_root(path.parent)
        if parent_module is None: return path
        else: return parent_module

In [46]:
find_module_root(Path(fbdev.__path__[0]) / 'utils.py')

Path('/Users/lukastk/proj_dev/fbdev/fbdev')

In [47]:
#|hide
show_doc(fbdev.utils.__get_module_path_hierarchy)

---

### __get_module_path_hierarchy

>      __get_module_path_hierarchy (path, hierarchy)

In [48]:
#|exporti
def __get_module_path_hierarchy(path, hierarchy):
    path = Path(path)
    if not path.exists(): raise FileNotFoundError(f"No file or directory found at: {path}")
    if path.is_file():
        if path.suffix != '.py': raise ValueError(f"File '{path}' is not a python file.")
        is_in_module = '__init__.py' in [p.parts[-1] for p in path.parent.glob('*')]
        if is_in_module:
            module_name = path.stem
            hierarchy.append((module_name, path))
            __get_module_path_hierarchy(path.parent, hierarchy)
    else:
        is_module = '__init__.py' in [p.parts[-1] for p in path.glob('*')]
        if is_module:
            module_name = path.stem
            hierarchy.append((module_name, path))
            __get_module_path_hierarchy(path.parent, hierarchy)
        
def get_module_path_hierarchy(path):
    hierarchy = []
    __get_module_path_hierarchy(path, hierarchy)
    return hierarchy

In [49]:
get_module_path_hierarchy(Path(fbdev.__path__[0]) / 'utils.py')

[('utils', Path('/Users/lukastk/proj_dev/fbdev/fbdev/utils.py')),
 ('fbdev', Path('/Users/lukastk/proj_dev/fbdev/fbdev'))]

In [50]:
#|hide
show_doc(fbdev.utils.get_function_from_py_file)

---

### get_function_from_py_file

>      get_function_from_py_file (file_path, func_name=None, args=[],
>                                 is_async=False)

In [51]:
#|export
def get_function_from_py_file(file_path, func_name=None, args=[], is_async=False):
    file_path = Path(file_path)
    module_path = find_module_root(file_path)
    is_in_module = module_path is not None
    
    # Check if the file exists
    if not file_path.is_file():
        raise ValueError(f"Not a file: {file_path}")
    if not file_path.exists():
        raise FileNotFoundError(f"No file found at: {file_path}")
    
    if func_name is None:
        func_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Read the contents of the file
    with open(file_path, 'r') as file:
        func_body_code = file.read()
        
    if not func_body_code.strip(): func_body_code = 'pass'
    
    # Tabify
    func_body_code = '\n'.join(list(map(lambda line: f"    {line}", func_body_code.split('\n'))))
    func_code = f"{'async ' if is_async else ''}def {func_name}({', '.join(args)}):\n{func_body_code}"
    
    if is_in_module:
        # This all is necessary to allow for relative imports in the code
        sys.path.insert(0, module_path.parent.absolute().as_posix())
        module_hierarchy = get_module_path_hierarchy(file_path)
        module_hierarchy_str = '.'.join([e[0] for e in reversed(module_hierarchy)])
        module_spec = importlib.util.spec_from_file_location(module_hierarchy_str, file_path.absolute().as_posix())
        code_module = importlib.util.module_from_spec(module_spec)
        locals_dict = code_module.__dict__
    else:
        locals_dict = {}
        
    exec(func_code, locals_dict)
    if is_in_module: sys.path.pop(0)
    
    return locals_dict[func_name]

In [52]:
#|hide
show_doc(fbdev.utils.SingletonMeta)

---

### SingletonMeta

>      SingletonMeta (name, bases=(), dct=None)

*type(object) -> the object's type
type(name, bases, dict, **kwds) -> a new type*

In [53]:
#|export
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

    def __new__(metacls, name, bases=(), dct=None):
        if dct is None:
            dct = {}
        return super().__new__(metacls, name, bases, dct)

In [ ]:
#|hide
show_doc(fbdev.utils.is_valid_python_var_name)

In [54]:
#|export
def is_valid_python_var_name(name: str) -> bool:
    """
    Check if the provided string is a valid Python variable name.
    
    Parameters:
    name (str): The string to check.
    
    Returns:
    bool: True if the string is a valid Python variable name, False otherwise.
    """
    # Check if the name is a Python keyword
    if keyword.iskeyword(name):
        return False
    
    # Regular expression to match valid Python identifiers
    valid_identifier_pattern = r'^[A-Za-z_][A-Za-z0-9_]*$'
    
    # Use the regular expression to check the validity of the variable name
    if re.match(valid_identifier_pattern, name):
        return True
    else:
        return False

In [58]:
# Example usage:
assert is_valid_python_var_name("my_var")
assert not is_valid_python_var_name("2var")
assert not is_valid_python_var_name("def")
assert not is_valid_python_var_name("my.var")
assert not is_valid_python_var_name("my.var!")